In [1]:
import pickle
import pandas
import numpy as np

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape,LSTM
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras import regularizers
from keras import backend as K

from sklearn.metrics import mean_squared_error

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X = np.load("train_X_normalized_lstm_delta.pkl.npy")
Y = np.load("train_Y_normalized_lstm_delta.pkl.npy")
reg_Y = np.load("train_Y_normalized_lstm_regression.pkl.npy")
# X = np.load("train_X_normalized_lstm_classification.pkl.npy")
# Y = np.load("train_Y_normalized_lstm_classification.pkl.npy")

In [3]:
X.shape

(778302, 60, 5)

In [4]:
X = X.reshape(X.shape[0], X.shape[1], -1)

In [5]:
Y.shape

(778302, 8)

In [ ]:
def mse_pred(y_reg):
    def fn(y_true, y_pred):
        categories = [-0.15, -0.075, -0.03,-0.005,0.005,0.03,0.075,0.15]
        pred = categories[np.argmax(y_pred)]
        return mean_squared_error(y_reg,pred)
    return fn

In [14]:
def build_model(X):
    model = Sequential()

    model.add(LSTM(512,
        input_shape=(X.shape[1],X.shape[2]),
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        512,
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(8, activation = 'softmax'))
    #Adm = keras.optimizers.Adam(lr = 0.001)
    model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=['accuracy','recall',mse_pred(input_dim = 1)])
    # K.tensorflow_backend._get_available_gpus()
    return model


In [15]:
model = build_model(X)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 60, 512)           1060864   
_________________________________________________________________
dropout_3 (Dropout)          (None, 60, 512)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 4104      
Total params: 3,164,168
Trainable params: 3,164,168
Non-trainable params: 0
_________________________________________________________________


In [18]:
from keras.models import load_model
model = load_model('lstm_delta.h5')
res = model.fit(X, Y,batch_size=512, epochs=40,verbose=1, shuffle=True,validation_split = 0.2)
model.save("lstm_delta.h5")

Train on 622641 samples, validate on 155661 samples
Epoch 1/40
622641/622641 [==============================] - 851s 1ms/step - loss: 1.8726 - acc: 0.3270 - val_loss: 1.9622 - val_acc: 0.2813
Epoch 2/40
622641/622641 [==============================] - 849s 1ms/step - loss: 1.8684 - acc: 0.3273 - val_loss: 1.9634 - val_acc: 0.2795
Epoch 3/40
622641/622641 [==============================] - 849s 1ms/step - loss: 1.8627 - acc: 0.3287 - val_loss: 1.9318 - val_acc: 0.2860
Epoch 4/40
622641/622641 [==============================] - 850s 1ms/step - loss: 1.8589 - acc: 0.3300 - val_loss: 1.9208 - val_acc: 0.2872
Epoch 5/40
622641/622641 [==============================] - 849s 1ms/step - loss: 1.8569 - acc: 0.3302 - val_loss: 1.9200 - val_acc: 0.2888
Epoch 6/40
622641/622641 [==============================] - 850s 1ms/step - loss: 1.8553 - acc: 0.3302 - val_loss: 1.9328 - val_acc: 0.2893
Epoch 7/40
622641/622641 [==============================] - 850s 1ms/step - loss: 1.8542 - acc: 0.3304 - val

KeyboardInterrupt: 

In [19]:
model.save("lstm_delta.h5")